In [2]:
import numpy
import nn
import idx2numpy

In [3]:
train_img = (idx2numpy.convert_from_file("train-images.idx3-ubyte")/255).reshape(-1, 28 * 28)
train_lbl = idx2numpy.convert_from_file("train-labels.idx1-ubyte")
test_img = (idx2numpy.convert_from_file("t10k-images.idx3-ubyte")/255).reshape(-1, 28 * 28)
test_lbl = idx2numpy.convert_from_file("t10k-labels.idx1-ubyte")

In [4]:
train_img.shape, test_img.shape

((60000, 784), (10000, 784))

In [13]:
m = nn.model()

m.add(nn.Layer_Dense(28 * 28, 500))
m.add(nn.Activation_ReLU())
m.add(nn.Layer_Dense(500, 10))


m.set(loss = nn.Loss_CategoricalCrossentropy(), 
      optimizer = nn.Optimizer_SGD(learning_rate = 0.8, decay= 1e-3))


m.train(train_img, train_lbl, epoches = 10000, print_every = 100, batch = 123)

Epoches = 100,  Loss = 9.101, Learning_rate = 0.728, Acc = 0.129
Epoches = 200,  Loss = 9.355, Learning_rate = 0.667, Acc = 0.102
Epoches = 300,  Loss = 9.678, Learning_rate = 0.616, Acc = 0.094


KeyboardInterrupt: 